# Preprocesamiento de Datos

En este notebook se lleva a cabo el proceso de preprocesamiento del dataset **Car Sales**, que servirá como entrada para el modelo de Machine Learning utilizado en el proyecto.  
El objetivo principal es dejar los datos en un formato limpio, coherente y totalmente utilizable para la fase de modelado.

Este preprocesamiento incluye:

- Inspección inicial del dataset  
- Tratamiento de valores faltantes  
- Normalización y estandarización de variables numéricas  
- Codificación de variables categóricas  
- Eliminación de duplicados  
- Guardado del dataset procesado



## 1. Carga del Dataset

Comenzamos importando las librerías necesarias y cargando el dataset original almacenado en la carpeta `/data`.  
Se realiza también una inspección básica para entender los tipos de datos y el número de valores faltantes.



In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [ ]:
# Subir dos niveles desde notebooks/ hasta la raíz del repo
repo_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

# Ruta al dataset original
data_path = os.path.join(repo_root, "data", "car_sales_dataset.csv")

# Cargar dataset
df = pd.read_csv(data_path)

df.head()

In [ ]:
df.info()
df.isna().sum()

## 2. Tratamiento de Valores Faltantes

Se revisan las columnas que contienen valores nulos y se aplican diferentes técnicas de imputación:

- Para variables numéricas: **mediana**
- Para variables categóricas: **moda**

Estas estrategias permiten mantener la distribución de los datos sin introducir sesgos significativos.


In [ ]:
# Imputaciones numéricas y categóricas
df = df.copy()

engine_median = df["Engine HP"].median()
df["Engine HP"] = df["Engine HP"].fillna(engine_median)

doors_mode = df["Number of Doors"].mode()[0]
df["Number of Doors"] = df["Number of Doors"].fillna(doors_mode)

## 3. Eliminación de Duplicados

Para garantizar la calidad del dataset, se eliminan todas las filas duplicadas.  
Esto ayuda a evitar que el modelo aprenda patrones erróneos debido a repeticiones en los datos.


In [ ]:
df = df.drop_duplicates().reset_index(drop=True)
df.shape

## 4. Limpieza y Normalización de Nombres de Columnas

Con el objetivo de mantener un estilo de nombres consistente y compatible con librerías de Machine Learning, se:

- eliminan espacios
- convierten los nombres a minúsculas
- sustituyen espacios por guiones bajos

Esto facilita el manejo posterior del dataset en el pipeline.


In [ ]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
df.head()

In [ ]:
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
num_cols

## 5. Normalización de Variables Numéricas

Para las variables numéricas se aplica `StandardScaler`, que transforma cada variable para que tenga:

- media = 0  
- desviación estándar = 1  

Esto es especialmente útil para modelos basados en distancia o gradiente.


In [ ]:
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

## 6. Codificación de Variables Categóricas

Las variables categóricas se convierten en variables numéricas mediante **One-Hot Encoding**.  
Esta técnica crea nuevas columnas binarizadas, permitiendo al modelo interpretar correctamente los valores categóricos.

Se utiliza la opción `drop_first=True` para evitar multicolinealidad.


In [ ]:
cat_cols = df.select_dtypes(include=["object"]).columns.tolist()
cat_cols

In [ ]:
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)
df.head()

In [ ]:
processed_dir = os.path.join(repo_root, "data", "processed")
os.makedirs(processed_dir, exist_ok=True)

output_path = os.path.join(processed_dir, "car_sales_processed.csv")
df.to_csv(output_path, index=False)

output_path